## Run your Llama Stack distribution

Nuke any old distro config files you might have lying around (I find these get in the way whenever I change my `.env` variables): 
```bash
ls ~/.llama/distributions/
rm -r ~/.llama/distributions/<name-of-your-distro>
```

Then, run your llama stack server with:
```bash
dotenv run uv run llama stack run distribution/run.yaml
```




## Setup and Imports


In [1]:
# Install dev packages if not already installed
# !uv pip install -e ".[dev]"

# Then, you will also need:
# uv pip install sentence-transformers
# uv pip install langchain-text-splitters

import json
import os
from datetime import datetime

import pandas as pd
from datasets import Dataset, load_dataset
from langchain.chat_models import BaseChatModel, init_chat_model
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_stack_client import LlamaStackClient
from rich.pretty import pprint
from tqdm import tqdm

from llama_stack_provider_ragas.constants import PROVIDER_ID_INLINE, PROVIDER_ID_REMOTE

/Users/dmanilof/src/llama-stack-provider-ragas/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Llama Stack Client Setup

- Make sure we have an inference model (model_type='llm')
- Make sure we have an embedding model (model_type='embedding')


In [2]:
# If using the remote provider, you will need ngrok to enable remote access to your Llama Stack server
# Otherwise, the base_url is just http://localhost:8321

client = LlamaStackClient(base_url=os.getenv("KUBEFLOW_LLAMA_STACK_URL"))
available_models = client.models.list()
pprint(available_models)

INFO:httpx:HTTP Request: GET https://36cd37c93429.ngrok-free.app/v1/models "HTTP/1.1 200 OK"


[
│   Model(
│   │   id='ollama/all-minilm:latest',
│   │   created=1769184594,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'embedding',
│   │   │   'provider_id': 'ollama',
│   │   │   'provider_resource_id': 'all-minilm:latest',
│   │   │   'embedding_dimension': 384
│   │   },
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/granite3.3:2b',
│   │   created=1769184594,
│   │   owned_by='llama_stack',
│   │   custom_metadata={'model_type': 'llm', 'provider_id': 'ollama', 'provider_resource_id': 'granite3.3:2b'},
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/granite4:latest',
│   │   created=1769184594,
│   │   owned_by='llama_stack',
│   │   custom_metadata={'model_type': 'llm', 'provider_id': 'ollama', 'provider_resource_id': 'granite4:latest'},
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/nomic-embed-text:latest',
│   │   created=1769184594,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'embedding',
│   │   │   'provider_id': 'ollama',
│   │   │   'provider_resource_id': 'nomic-embed-text:latest',
│   │   │   'embedding_dimension': 768,
│   │   │   'context_length': 8192
│   │   },
│   │   object='model'
│   ),
│   Model(
│   │   id='ollama/all-minilm:l6-v2',
│   │   created=1769184594,
│   │   owned_by='llama_stack',
│   │   custom_metadata={
│   │   │   'model_type': 'embedding',
│   │   │   'provider_id': 'ollama',
│   │   │   'provider_resource_id': 'all-minilm:l6-v2',
│   │   │   'embedding_dimension': 384,
│   │   │   'context_length': 512
│   │   },
│   │   object='model'
│   )
]

## Setup a simple RAG system using LangChain

NOTE: an alternative here is to use Llama Stack's client.vector_stores.create(), see: https://llamastack.github.io/docs/building_applications/rag, however that seems to work only with the files API, not the datasets API. Since our dataset is already in Hugging Face, the LangChain approach seems a bit more natural.

### Datasets we will use

- We will use `m-ric/huggingface_doc` as our knowledge base (the Hugging Face documentation).
- We will use `m-ric/huggingface_doc_qa_eval` as our ground truth for evaluation (questions and the expected answers).
- We will create `dmaniloff/hf_doc_qa_ragas_eval_dataset` as our evaluation dataset, which will be the result of using our RAG system to answer questions about the Hugging Face documentation.


In [3]:
# This is a datset of the Hugging Face documentation
dataset_name = "m-ric/huggingface_doc"
text_column = "text"

loader = HuggingFaceDatasetLoader(dataset_name, text_column)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
chunks = text_splitter.split_documents(documents)
print(f"Split blog post into {len(chunks)} sub-documents.")

Split blog post into 29129 sub-documents.


In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = InMemoryVectorStore(embeddings)
document_ids = vector_store.add_documents(documents=chunks[:10_000])  # make it quick
print(document_ids[:3])

/var/folders/8g/k40y0tk10_g28ck0h9nxph9m0000gn/T/ipykernel_26737/3100604079.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


['e98474b4-6138-493b-8ea6-942b5e009da5', '24b066e7-5081-43cf-a01a-bdd4f46510b7', '5f033f92-1301-40c9-a657-2265f9c67a50']


### Simple RAG System Definition


In [5]:
rag_model = init_chat_model(model="granite3.3:2b", model_provider="ollama")
rag_model.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'granite3.3:2b', 'created_at': '2026-01-23T16:10:38.083543Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 668728375, 'load_duration': 406257542, 'prompt_eval_count': 47, 'prompt_eval_duration': 146383875, 'eval_count': 10, 'eval_duration': 115286791}, id='lc_run--019beb9f-31e4-7a72-a34d-9044d43e3a41-0', tool_calls=[], invalid_tool_calls=[])

In [6]:
def simple_rag_chain(
    model: BaseChatModel,
    vector_store: InMemoryVectorStore,
    query: str,
):
    """Inject retrieved documents into the system message."""
    retrieved_docs = vector_store.similarity_search(query)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )
    return {
        "answer": model.invoke(system_message).content,
        "sources": [d.metadata for d in retrieved_docs],
    }


In [7]:
# test it
simple_rag_chain(
    rag_model, vector_store, "How do I create a dataset card on Hugging Face?"
)

/Users/dmanilof/src/llama-stack-provider-ragas/.venv/lib/python3.12/site-packages/langchain_core/vectorstores/utils.py:92: RuntimeWarning: overflow encountered in dot
  similarity = np.dot(x, y.T) / np.outer(x_norm, y_norm)


{'answer': "The corpus was created by utilizing the Hugging Face Hub, which is a platform for sharing datasets and machine learning models. To contribute effectively, it's essential to provide comprehensive information in a *README.md* file within each dataset repository on the Hub. This document should include metadata tags in YAML format using the [`datasets-tagging` application](https://huggingface.co/datasets/tagging/) for better searchability and discoverability by other community members.\n\nFollow these steps to create your *README.md* file:\n\n1. Clone the `datasets-tagging` repository and run it locally using YAML tags in appropriate categories.\n2. Refer to the [\\ud83e\\udd17 Datasets guide](https://github.com/huggingface/datasets/blob/master/templates/README_guide.md) for creating informative dataset cards, using a template like the one provided in the `lewtun/github-issues` repository.\n3. Fill out your *README.md* with necessary details about your dataset, including its c

## Evaluation Data Setup

### Prepare Evaluation Dataset
Here we prepare an evaluation dataset by asking the RAG system to answer questions
from the Hugging Face documentation and placing them side-by-side with the ground truth.

In [8]:
# This is a dataset of questions and answers using the Hugging Face documentation
# as a source.
ground_truth = load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

evaluation_data = []
for row in tqdm(ground_truth.select(range(10))):
    question = row["question"]
    expected_answer = row["answer"]
    rag_result = simple_rag_chain(rag_model, vector_store, question)
    model_response = rag_result.get("answer", "")
    evaluation_data.append(
        {
            "user_input": question,  # ragas expects "user_input" as the key for the question
            "reference": expected_answer,  # ragas expects "reference" as the key for the ground truth answer
            "response": model_response,
        }
    )


100%|██████████| 10/10 [01:53<00:00, 11.34s/it]


In [9]:
pprint(evaluation_data[:2])

[
│   {
│   │   'user_input': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n',
│   │   'reference': 'x86_64-unknown-linux-musl',
│   │   'response': "The provided context lists several binaries for the `tokenizers` library, each tailored for different operating systems and architectures:\n\n1. `tokenizers-linux-arm64-musl`: This is the AARCH64 (aarch64) binary compiled with the musl C library on Linux. It's optimized for ARM64 architecture systems using the musl C library.\n\n2. `tokenizers-win32-x64-msvc`: This is the x86_64 binary compiled with Microsoft Visual C++ (MSVC) on Windows. It's designed for 64-bit Windows systems using the MSVC compiler.\n\n3. `tokenizers-freebsd-x64`: This is the x86_64 binary compiled for FreeBSD using the LLVM/Clang compiler suite. It's optimized for FreeBSD 12+ (x64) systems.\n\n4. `tokenizers-win32-ia32-msvc`: This is the i686 binary compiled with Microsoft Visual C++ (MSVC) on Windows. It's designed for 32-bit Windows systems using the MSVC compiler, specifically targeting IA32 architecture.\n\nThese binaries allow users to integrate the `tokenizers` library into their respective operating system environments and architectures."
│   },
│   {
│   │   'user_input': 'What is the purpose of the BLIP-Diffusion model?\n',
│   │   'reference': 'The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.',
│   │   'response': "Diffusion models are machine learning systems trained to gradually remove Gaussian noise from random inputs, such as images, until they reach a desired sample type. They work by performing a series of denoising steps sequentially, which can be computationally expensive and memory-intensive due to their operation in the pixel space, especially for high-resolution images.\n\nLatent diffusion is an advancement that addresses these challenges by applying the diffusion process in a lower dimensional _latent_ space instead of pixel space. This approach reduces memory consumption and computational complexity significantly. For more detailed information on how diffusion models work and their improvements to latent diffusion, refer to [Stability AI's announcement](https://stability.ai/blog/stable-diffusion-announcement) and our corresponding [blog post](https://huggingface.co/blog/stable_diffusion#how-does-stable-diffusion-work).\n\nThe original codebase for Stable Diffusion v1.0 can be found at [CompVis/stable-diffusion](https://github.com/CompVis/stable-diffusion), and Stable Diffusion v2.0 is available at [Stability-AI/stablediffusion](https://github.com/Stability-AI/stablediffusion). Official checkpoints for these versions and various tasks can be accessed through the CompVis, Runway, and Stability AI Hub organizations on Hugging Face:\n\n- Annotated Diffusion Model: [huggingface/diffusers](https://github.com/huggingface/diffusers)\n- Multi-Concept Customization of Text-to-Image Diffusion: [Multi-Concept Customization of Text-to-Image Diffusion](https://www.cs.cmu.edu/~custom-diffusion/)\n\n### Citation:\n```\n@article{patil2022stable,\\n  author = {Patil, Suraj and Cuenca, Pedro and Lambert, Nathan and von Platen, Patrick},\\n  title = {Stable Diffusion with \\ud83e\\udde8 Diffusers},\\n  journal = {Hugging Face Blog},\\n  year = {2022},\\n  note = {[https://huggingface.co/blog/rlhf](https://huggingface.co/blog/stable_diffusion)},\\n}\n```"
│   }
]

In [10]:
ragas_eval_dataset = Dataset.from_list(evaluation_data)
ragas_eval_dataset.push_to_hub("dmaniloff/hf_doc_qa_ragas_eval_dataset")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1249.79ba/s]
Processing Files (1 / 1): 100%|██████████| 20.2kB / 20.2kB, 33.8kB/s  
New Data Upload: 100%|██████████| 20.2kB / 20.2kB, 33.8kB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.68s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/dmaniloff/hf_doc_qa_ragas_eval_dataset/commit/320679d955cec84e1a4beec545bcd269a0b79ef6', commit_message='Upload dataset', commit_description='', oid='320679d955cec84e1a4beec545bcd269a0b79ef6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/dmaniloff/hf_doc_qa_ragas_eval_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='dmaniloff/hf_doc_qa_ragas_eval_dataset'), pr_revision=None, pr_num=None)

### Use the evaluation dataset in our Llama Stack distribution
The correct way to do this is to pre-register the dataset in our Llama Stack distribution. However there seems to be a mismatch between the stack expecting RegisterDatasetRequest but the actual provider implementations still use the old Dataset signature.

For this reason, we will use the deprecated (but still available) `register_dataset` method.

#### [not working] Pre-register the newly uploaded dataset in our Llama Stack distribution.

Add to your stack config's registered_resources.datasets section:

```yaml
  registered_resources:
    datasets:
    - purpose: eval/messages-answer
      source:
        type: uri
        uri: huggingface://datasets/your-org/your-dataset?split=train
      metadata: {}
      dataset_id: your-dataset-id
```

Your config needs the huggingface datasetio provider (lines 89-94):

```yaml
  providers:
    datasetio:
    - provider_id: huggingface
      provider_type: remote::huggingface
      config:
        kvstore:
          namespace: datasetio::huggingface
          backend: kv_default
```


In [22]:
dataset = client.beta.datasets.register(
    purpose="eval/messages-answer",
    source={
        "type": "uri",
        "uri": "huggingface://datasets/dmaniloff/hf_doc_qa_ragas_eval_dataset?split=train",
    },
    dataset_id="hf_doc_qa_ragas_eval",
)

/var/folders/8g/k40y0tk10_g28ck0h9nxph9m0000gn/T/ipykernel_26737/857061480.py:1: DeprecationWarning: deprecated
  dataset = client.beta.datasets.register(
INFO:httpx:HTTP Request: POST https://36cd37c93429.ngrok-free.app/v1beta/datasets "HTTP/1.1 200 OK"


In [23]:
datasets = client.beta.datasets.list()
pprint(datasets)

INFO:httpx:HTTP Request: GET https://36cd37c93429.ngrok-free.app/v1beta/datasets "HTTP/1.1 200 OK"


[
│   DatasetListResponseItem(
│   │   identifier='hf_doc_qa_ragas_eval',
│   │   provider_id='huggingface',
│   │   purpose='eval/messages-answer',
│   │   source=DatasetListResponseItemSourceUriDataSource(
│   │   │   uri='huggingface://datasets/dmaniloff/hf_doc_qa_ragas_eval_dataset?split=train',
│   │   │   type='uri'
│   │   ),
│   │   metadata={},
│   │   provider_resource_id='hf_doc_qa_ragas_eval',
│   │   type='dataset'
│   )
]

### Benchmark Registration

Register a benchmark that defines what metrics to use for evaluation.


In [24]:
# comment out the provider you don't want to run
benchmarks_providers = [
    ("ragas_demo_benchmark__inline", PROVIDER_ID_INLINE),
    ("ragas_demo_benchmark__remote", PROVIDER_ID_REMOTE),
]

for benchmark_id, provider_id in benchmarks_providers:
    benchmark_response = client.alpha.benchmarks.register(
        benchmark_id=benchmark_id,
        dataset_id="hf_doc_qa_ragas_eval",
        scoring_functions=[
            # "answer_relevancy",  # How relevant is the answer to the question?
            # "context_precision",     # How precise are the retrieved contexts?
            # "faithfulness",          # How faithful is the answer to the contexts?
            # "context_recall",        # How much of the ground truth is covered by contexts?
            "semantic_similarity"  # How similar is the answer to the ground truth? The metric's class is called AnswerSimilarity, but the metric's name is semantic_similarity (I know).
        ],
        provider_id=provider_id,
        # metadata={
        #     "provider": "ragas",
        #     "version": "1.0",
        #     "metrics_count": len(ragas_metrics),
        #     "created_at": datetime.now().isoformat()
        # }
    )

pprint(benchmark_response)

/var/folders/8g/k40y0tk10_g28ck0h9nxph9m0000gn/T/ipykernel_26737/469531212.py:8: DeprecationWarning: deprecated
  benchmark_response = client.alpha.benchmarks.register(
INFO:httpx:HTTP Request: POST https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"


None

In [25]:
benchmarks = client.alpha.benchmarks.list()
pprint(benchmarks)

INFO:httpx:HTTP Request: GET https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"


[
│   Benchmark(
│   │   dataset_id='hf_doc_qa_ragas_eval',
│   │   identifier='ragas_demo_benchmark__inline',
│   │   provider_id='trustyai_ragas_inline',
│   │   scoring_functions=['semantic_similarity'],
│   │   metadata={},
│   │   provider_resource_id='ragas_demo_benchmark__inline',
│   │   type='benchmark'
│   ),
│   Benchmark(
│   │   dataset_id='hf_doc_qa_ragas_eval',
│   │   identifier='ragas_demo_benchmark__remote',
│   │   provider_id='trustyai_ragas_remote',
│   │   scoring_functions=['semantic_similarity'],
│   │   metadata={},
│   │   provider_resource_id='ragas_demo_benchmark__remote',
│   │   type='benchmark'
│   )
]

## Evaluation Execution

Run the evaluation using our Ragas out-of-tree provider.


In [26]:
remote_job = client.alpha.eval.run_eval(
    benchmark_id="ragas_demo_benchmark__remote",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite4:latest",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(remote_job)

INFO:httpx:HTTP Request: POST https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='b2b92eea-2257-42a0-b8dc-75bfd7c6fa18',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite4:latest',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark__remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark__remote',
│   │   │   'provider_id': 'trustyai_ragas_remote',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'hf_doc_qa_ragas_eval',
│   │   │   'scoring_functions': ['semantic_similarity'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-ragas-eval-dspa-ragas-eval-pipeline.apps.rosa.diego-ragas.9s6r.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-pipeline',
│   │   │   'llama_stack_url': 'https://36cd37c93429.ngrok-free.app',
│   │   │   'base_image': 'registry.redhat.io/ubi9/python-312:latest@sha256:55e489965ae3bcd954d33a7022c8a4af4774ac2cdfd8a4011f1f28ae3b7b8f8f',
│   │   │   'pipelines_api_token': None
│   │   }
│   },
│   kubeflow_run_id='5f7bb478-4b05-4a1a-8984-5284201a3a16',
│   result=None
)

In [27]:
# Review settings in distributinon/run.yaml, eg., note that
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file(all-MiniLM-L6-v2)

inline_job = client.alpha.eval.run_eval(
    benchmark_id="ragas_demo_benchmark__inline",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite4:latest",  # this is the model we're using as a judge for the evaluation
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(inline_job)

INFO:httpx:HTTP Request: POST https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='0',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

## Results Display


In [ ]:
# wait a bit for the job to complete
pprint(
    client.alpha.eval.jobs.status(
        benchmark_id="ragas_demo_benchmark__inline", job_id=inline_job.job_id
    )
)

INFO:httpx:HTTP Request: GET https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs/0 "HTTP/1.1 200 OK"


Job(
│   job_id='0',
│   status='completed',
│   result={
│   │   'generations': [
│   │   │   {
│   │   │   │   'user_input': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n',
│   │   │   │   'response': "The provided context lists several binaries for the `tokenizers` package, each tailored for specific operating systems and architectures:\n\n1. `tokenizers-linux-arm64-musl`: This is a binary specifically designed for Linux systems with an aarch64 (ARM64) architecture using the musl C library.\n2. `tokenizers-win32-x64-msvc`: A Windows-based binary, optimized for x86_64 (amd64) architecture under Microsoft's Visual Studio compiler (msvc).\n3. `tokenizers-freebsd-x64`: A binary designed for FreeBSD systems with an x86_64 (amd64) architecture.\n4. `tokenizers-win32-ia32-msvc`: Another Windows binary, but this one is optimized for i686 (ia32) architecture under Microsoft's Visual Studio compiler (msvc).\n\nThese binaries allow users to incorporate the `tokenizers` package into their respective systems and application environments, ensuring compatibility with their specific architectures and operating systems.",
│   │   │   │   'reference': 'x86_64-unknown-linux-musl'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is the purpose of the BLIP-Diffusion model?\n',
│   │   │   │   'response': 'Diffusion models are a type of machine learning system trained to gradually remove random Gaussian noise from raw data, such as images, to generate a sample of interest. They work by iteratively refining an initial noisy input through a series of denoising steps until the desired output is achieved. This process can be computationally expensive and memory-intensive due to its sequential nature and reliance on pixel space.\n\nLatent diffusion models address these limitations by operating over a lower-dimensional latent space instead of the actual pixel space. This approach reduces memory usage and computational complexity, making it easier to train and use for inference. Stable Diffusion is an example of such a model, which has shown impressive results in image generation and offers various versions with differing capabilities through its codebase on GitHub.\n\nFor more detailed information about diffusion models, you can refer to [this Colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb) and the Stability AI announcement or our blog post for technical insights into how Stable Diffusion works. Explore available checkpoints on Hugging Face, Runway, and Stability AI Hubs to find the most suitable option for your needs.\n\nCitation: Patil et al., 2022 - "Stable Diffusion with \\ud83e\\udde8 Diffusers" [Hugging Face Blog](https://huggingface.co/blog/stable_diffusion)',
│   │   │   │   'reference': 'The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'How can a user claim authorship of a paper on the Hugging Face Hub?\n',
│   │   │   │   'response': "The Hugging Face Hub is a powerful platform designed to facilitate the sharing and collaboration of machine learning models, datasets, and research in natural language processing (NLP). It's built on the principles of openness, transparency, and community involvement. Here are some key features:\n\n1. **Model Sharing**: Users can upload their trained models, making them accessible to a vast community of data scientists and researchers. This fosters collaboration and accelerates machine learning innovation. You can learn more about sharing Transformers models on the Hub [here](https://huggingface.co/docs/transformers/model_sharing).\n\n2. **Dataset Sharing**: Hugging Face allows users to share datasets, promoting data transparency and reproducibility. By following the guidelines for uploading datasets, you can contribute to a rich repository of diverse NLP resources [here](https://huggingface.co/docs/

In [ ]:
# wait a bit for the job to complete
pprint(
    client.alpha.eval.jobs.status(
        benchmark_id="ragas_demo_benchmark__remote", job_id=remote_job.job_id
    )
)

INFO:httpx:HTTP Request: GET https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs/dc712f2b-b849-4ba1-a57f-ef1860e012f5 "HTTP/1.1 200 OK"


Job(
│   job_id='dc712f2b-b849-4ba1-a57f-ef1860e012f5',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite4:latest',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark__remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark__remote',
│   │   │   'provider_id': 'trustyai_ragas_remote',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'hf_doc_qa_ragas_eval',
│   │   │   'scoring_functions': ['semantic_similarity'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-ragas-eval-dspa-ragas-eval-pipeline.apps.rosa.diego-ragas.9s6r.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-pipeline',
│   │   │   'llama_stack_url': 'https://36cd37c93429.ngrok-free.app',
│   │   │   'base_image': 'registry.redhat.io/ubi9/python-312:latest@sha256:55e489965ae3bcd954d33a7022c8a4af4774ac2cdfd8a4011f1f28ae3b7b8f8f',
│   │   │   'pipelines_api_token': '**********'
│   │   }
│   },
│   kubeflow_run_id='49bea157-7ed4-41d5-b5c4-964ce08060c6',
│   result=None
)

In [ ]:
remote_results = client.alpha.eval.jobs.retrieve(
    benchmark_id="ragas_demo_benchmark__remote", job_id=remote_job.job_id
)
pprint(remote_results)

INFO:httpx:HTTP Request: GET https://24700eefcd22.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs/6f8d9ed6-ca9f-451a-ae8e-80d005d368f6/result "HTTP/1.1 200 OK"


EvaluateResponse(generations=[], scores={})

In [ ]:
inline_results = client.alpha.eval.jobs.retrieve(
    benchmark_id="ragas_demo_benchmark__inline", job_id=inline_job.job_id
)
pprint(inline_results)

INFO:httpx:HTTP Request: GET https://36cd37c93429.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs/0/result "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'user_input': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n',
│   │   │   'response': "The provided context lists several binaries for the `tokenizers` package, each tailored for specific operating systems and architectures:\n\n1. `tokenizers-linux-arm64-musl`: This is a binary specifically designed for Linux systems with an aarch64 (ARM64) architecture using the musl C library.\n2. `tokenizers-win32-x64-msvc`: A Windows-based binary, optimized for x86_64 (amd64) architecture under Microsoft's Visual Studio compiler (msvc).\n3. `tokenizers-freebsd-x64`: A binary designed for FreeBSD systems with an x86_64 (amd64) architecture.\n4. `tokenizers-win32-ia32-msvc`: Another Windows binary, but this one is optimized for i686 (ia32) architecture under Microsoft's Visual Studio compiler (msvc).\n\nThese binaries allow users to incorporate the `tokenizers` package into their respective systems and application environments, ensuring compatibility with their specific architectures and operating systems.",
│   │   │   'reference': 'x86_64-unknown-linux-musl'
│   │   },
│   │   {
│   │   │   'user_input': 'What is the purpose of the BLIP-Diffusion model?\n',
│   │   │   'response': 'Diffusion models are a type of machine learning system trained to gradually remove random Gaussian noise from raw data, such as images, to generate a sample of interest. They work by iteratively refining an initial noisy input through a series of denoising steps until the desired output is achieved. This process can be computationally expensive and memory-intensive due to its sequential nature and reliance on pixel space.\n\nLatent diffusion models address these limitations by operating over a lower-dimensional latent space instead of the actual pixel space. This approach reduces memory usage and computational complexity, making it easier to train and use for inference. Stable Diffusion is an example of such a model, which has shown impressive results in image generation and offers various versions with differing capabilities through its codebase on GitHub.\n\nFor more detailed information about diffusion models, you can refer to [this Colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb) and the Stability AI announcement or our blog post for technical insights into how Stable Diffusion works. Explore available checkpoints on Hugging Face, Runway, and Stability AI Hubs to find the most suitable option for your needs.\n\nCitation: Patil et al., 2022 - "Stable Diffusion with \\ud83e\\udde8 Diffusers" [Hugging Face Blog](https://huggingface.co/blog/stable_diffusion)',
│   │   │   'reference': 'The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.'
│   │   },
│   │   {
│   │   │   'user_input': 'How can a user claim authorship of a paper on the Hugging Face Hub?\n',
│   │   │   'response': "The Hugging Face Hub is a powerful platform designed to facilitate the sharing and collaboration of machine learning models, datasets, and research in natural language processing (NLP). It's built on the principles of openness, transparency, and community involvement. Here are some key features:\n\n1. **Model Sharing**: Users can upload their trained models, making them accessible to a vast community of data scientists and researchers. This fosters collaboration and accelerates machine learning innovation. You can learn more about sharing Transformers models on the Hub [here](https://huggingface.co/docs/transformers/model_sharing).\n\n2. **Dataset Sharing**: Hugging Face allows users to share datasets, promoting data transparency and reproducibility. By following the guidelines for uploading datasets, you can contribute to a rich repository of diverse NLP resources [here](https://huggingface.co/docs/datasets/upload_dataset).\n\n3. **Documentation and Learning Resources**: Hugging Face offers compr

## Inline vs Remote Side-by-side

In [ ]:
pd.DataFrame.from_dict(
    {
        "inline": [
            r["score"] for r in inline_results.scores["answer_relevancy"].score_rows
        ],
        "remote": [
            r["score"] for r in remote_results.scores["answer_relevancy"].score_rows
        ],
    },
).assign(diff=lambda df: df["remote"] - df["inline"])